In [2]:
import pandas as pd
import numpy as np
from scipy.stats import t
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import os
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator
from sklearn.manifold import TSNE
import seaborn as sns

In [65]:
mz2_10 = pd.read_csv( "D:\\sar_time_series-fix_inference\\data\\final_otm.csv")

In [87]:
predictions = pd.read_csv("D:\\sar_time_series-fix_inference\\OTM\\LSTM\\predictions.csv", index_col='Date')

In [91]:
if predictions.shape[1] == 1:
    predictions.iloc[:, 0] = predictions.iloc[:, 0].astype(str)
    predictions_split = predictions.iloc[:, 0].str.split(',', expand=True)
    predictions_split.columns = mz2_10.columns[1:]
else:
    predictions_split = predictions
    predictions_split.columns = mz2_10.columns[1:]

result = pd.DataFrame()

for pid in mz2_10.columns[1:]:
    if pid in predictions_split.columns:
        actual_values = mz2_10[pid].tail(60).values
        predicted_values = predictions_split[pid].values
        actual_values = pd.to_numeric(actual_values, errors='coerce')
        predicted_values = pd.to_numeric(predicted_values, errors='coerce')
        
        if np.isnan(actual_values).any() or np.isnan(predicted_values).any():
            continue
    else:
        continue

    residuals = actual_values - predicted_values
    residuals_std = np.std(residuals)
    degrees_of_freedom = len(predicted_values) - 1
    margin_of_error = t.ppf(0.95, degrees_of_freedom) * residuals_std
    lower_bound = predicted_values - margin_of_error
    upper_bound = predicted_values + margin_of_error
    anomalies = (actual_values < lower_bound) | (actual_values > upper_bound)
    
    pid_result = pd.DataFrame({
        'pid': pid,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'actual_value': actual_values,
        'predicted_value': predicted_values,
        'is_anomaly': anomalies
    })
    
    result = pd.concat([result, pid_result], ignore_index=True)

output_path = 'D:\\sar_time_series-fix_inference\\OTM\\LSTM\\anomaly_otm_lstm_95.csv'
result.to_csv(output_path, index=False)

print(f"Results saved to {output_path}")

Results saved to D:\sar_time_series-fix_inference\OTM\LSTM\anomaly_otm_lstm_95.csv
